In [1]:
import numpy as np
import open3d as o3d
import csv
from sklearn.cluster import KMeans

In [2]:
lat = []
lon = []
alt = []
intensity = []

In [3]:
def LLA_to_XYZ(Lat,Lon,Alt,intensity):
    ''' Takes Latitude, Longitude and Altitdue and converts this data into
    x,y and z cartesion co-ordinates. Returns intensity as is.'''
    r_earth = (6378.137*1000)
    LatCos = np.cos(Lat*np.pi/180)
    LatSin = np.sin(Lat*np.pi/180)
    LonCos = np.cos(Lon*np.pi/180)
    LonSin = np.sin(Lon*np.pi/180)
    f = 1.0 / 298.257224
    C = 1.0/ np.sqrt(LatCos**2 + ((1-f)**2)*((LatSin)**2))
    S = ((1.0 - f)**2)*(C)
    x = (r_earth*C)*LatCos*LonCos
    y = (r_earth*C)*LatCos*LonSin
    z = Alt
    return x,y,z,intensity

In [4]:
def readfile(filename):
    ''' This function reads the file -"filename"- and appends 
    the data to the lat,lon,alt and intensity arrays '''
    global lat,lon,alt,intensity
    with open(filename) as csvfile:
        reader = csv.reader(csvfile, delimiter =' ')
        for row in reader:
            lat.append(float(row[0]))
            lon.append(float(row[1]))
            alt.append(float(row[2]))
            intensity.append(float(row[3]))

In [5]:
readfile("final_project_data/final_project_point_cloud.fuse")
points_xyzi = np.ones((len(lat),4))

In [6]:
for i in range(0,len(lat)):
    points_xyzi[i] = LLA_to_XYZ(lat[i],lon[i],alt[i],intensity[i])

In [7]:
print(points_xyzi.shape)

(430736, 4)


In [8]:
print(lat[31])
print(lon[31])
print(alt[31])
print(intensity[31])
print(points_xyzi[31])

45.90370042
11.02826286
226.2948
4.0
[4.36390869e+06 8.50492097e+05 2.26294800e+02 4.00000000e+00]


In [9]:
def write_point_cloud_object(array,name):
    ''' Takes array of point cloud (x,y,z) data and writes it to a 
    point cloud object file viewable in meshLab'''
    pc_object = open(name,"w")
    for j in array:
        pc_object.write("v "+str(j[0])+" "+str(j[1])+" "+str(j[2])+"\n")
    pc_object.close

In [10]:
write_point_cloud_object(points_xyzi,"All_data.obj")

In [11]:
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
X_scaled = preprocessing.scale(points_xyzi[ 0:400000 , 0:3])
X_norm = preprocessing.normalize(points_xyzi[:, 0:3], norm='l2')

#y_pred = DBSCAN(eps=0.3, min_samples=10).fit_predict(X_norm)
#y_pred = AgglomerativeClustering(n_clusters=2,affinity="euclidean",linkage="ward").fit_predict(X_norm)
y_pred = KMeans(n_clusters=2).fit_predict(points_xyzi[:, 2].reshape(-1,1))

/home/jordan/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [18]:
import collections
def planar_filter(xyzi, grid_size, lower_bound):

    scale = 1 / grid_size
    scale = int(scale) if scale > 1 else scale

    # Count height for each grid_size
    d = collections.defaultdict(list)
    xy = np.round(xyzi[:, 0:2] * scale)
    for idx in range(xyzi.shape[0]):
        x, y = xy[idx, :]
        z = xyzi[idx, 2]
        d[(x, y)].append((idx, z))

    # Filter the grid. Remove those with large different in z axis
    poi = []
    for xy, list_of_idx_and_z in d.items():
        list_idx, list_z = zip(*list_of_idx_and_z)
        if max(list_z) - min(list_z) >= lower_bound:
            poi.extend(list_idx)

    return poi

poi = planar_filter(points_xyzi, grid_size=0.5, lower_bound=6)

In [19]:
xyza_planar = points_xyzi[poi, :]
y_pred = KMeans(n_clusters=8).fit_predict(xyza_planar)

In [21]:
#Using open3d - creating open3d object
p = o3d.geometry.PointCloud()
# print(points_xyzi.shape)
a = points_xyzi[ : , 0:3][points_xyzi[ : , 2].argsort()]
p.points = o3d.utility.Vector3dVector(xyza_planar[ : , 0:3])
# help(o3d.geometry.PointCloud().colors)
colours = np.array([[255,255,0],[255,0,255],[0,255,255],[255,0,0],[0,255,0],[0,0,255],[128,255,0],[255,128,0],[0,128,255],[0,0,0],[255,255,0],[255,0,255],[0,255,255],[255,0,0],[0,255,0],[0,0,255],[128,255,0],[255,128,0],[0,128,255],[0,0,0]])
intens = np.zeros((np.size(points_xyzi[ : , 3]),3))
scaled_intensity = points_xyzi[ : , 3]/255
# intens[:,0] = np.reshape(scaled_intensity,-1)
# intens[:,0] = np.zeros(430736,)
# intens[:,1] = np.reshape(scaled_intensity,-1)
# intens[:,1] = np.zeros(430736,)
# # # intens[:,2] = np.reshape(scaled_intensity,-1)
for i in range(len(y_pred)):
    intens[i] = colours[y_pred[i]]
# intens[:150000,0] = np.zeros(150000,)
# intens[:150000,1] = np.zeros(150000,)
# intens[:150000,2] = np.ones(150000,)
# intens[150000:300000,1] = np.zeros(150000,)
# intens[150000:300000,2] = np.zeros(150000,)
# intens[150000:300000,0] = np.ones(150000,)
# intens[300000:,2] = np.zeros(430736-300000,)
# intens[300000:,0] = np.zeros(430736-300000,)
# intens[300000:,1] = np.ones(430736-300000,)


p.colors = o3d.utility.Vector3dVector(intens)
#p.colors = o3d.utility.Vector3dVector(points_xyzi[ : , 3])
o3d.io.write_point_cloud("./Alldata.ply", p)
o3d.visualization.draw_geometries([p])

Ethan splines here

In [17]:
print((intens[:,0]).shape)

(430736,)


In [43]:
#Downsampling by voxel
down_voxel = o3d.geometry.voxel_down_sample(p, voxel_size=0.8)
o3d.io.write_point_cloud("./Downsampled_voxel.ply", down_voxel)
# o3d.visualization.draw_geometries([down_voxel])


#Downsampling uniformly
down_uniform = o3d.geometry.uniform_down_sample(p, every_k_points=5)
o3d.io.write_point_cloud("./Downsampled_uniform.ply", down_uniform)
# o3d.visualization.draw_geometries([down_uniform])

#Downsampling statistical outlier removal
down_stats, ind = o3d.geometry.statistical_outlier_removal(down_voxel,nb_neighbors=50,std_ratio=5.0)
# inliers = o3d.geometry.select_down_sample(down_stats, ind)
o3d.io.write_point_cloud("./Downsampled_Removed_Stat_Outliers.ply", down_stats)
o3d.visualization.draw_geometries([down_stats])
print(np.size(p))

1


In [41]:
print(np.size(p.points))
print(np.size(down_voxel.points))
print(np.size(down_stats.points))

1292208
39780
39630


In [42]:
xyz_load = np.asarray(down_voxel.points)
print(xyz_load[0])

[4.36385641e+06 8.50503315e+05 2.25276640e+02]


In [ ]:
X_norm = preprocessing.normalize(xyz_load, norm='l2')

#y_pred = DBSCAN(eps=0.3, min_samples=10).fit_predict(X_norm)
#y_pred = AgglomerativeClustering(n_clusters=2,affinity="euclidean",linkage="ward").fit_predict(X_norm)
y_pred = KMeans(n_clusters=5).fit_predict(X_norm)

In [ ]:
np.size(xyz_load)

In [ ]:
#Using open3d - creating open3d object
p1 = o3d.geometry.PointCloud()

p1.points = o3d.utility.Vector3dVector(xyz_load)
# help(o3d.geometry.PointCloud().colors)
#colours = np.array([[255,255,0],[255,0,255],[0,255,255],[255,0,0],[0,255,0],[0,0,255],[128,255,0],[255,128,0],[0,128,255],[0,0,0],[255,255,0],[255,0,255],[0,255,255],[255,0,0],[0,255,0],[0,0,255],[128,255,0],[255,128,0],[0,128,255],[0,0,0]])
intens1 = np.zeros((np.size(xyz_load),3))
#scaled_intensity = points_xyzi[ : , 3]/255
# intens[:,0] = np.reshape(scaled_intensity,-1)
# intens[:,0] = np.zeros(430736,)
# intens[:,1] = np.reshape(scaled_intensity,-1)
# intens[:,1] = np.zeros(430736,)
# # intens[:,2] = np.reshape(scaled_intensity,-1)
# for i in range(len(y_pred)):
#     intens[i] = colours[y_pred[i]]
# intens[:150000,0] = np.zeros(150000,)
# intens[:150000,1] = np.zeros(150000,)
# intens[:150000,2] = np.ones(150000,)
# intens[150000:300000,1] = np.zeros(150000,)
# intens[150000:300000,2] = np.zeros(150000,)
# intens[150000:300000,0] = np.ones(150000,)
# intens[300000:,2] = np.zeros(430736-300000,)
# intens[300000:,0] = np.zeros(430736-300000,)
# intens[300000:,1] = np.ones(430736-300000,)


p1.colors = o3d.utility.Vector3dVector(intens1)
#p.colors = o3d.utility.Vector3dVector(points_xyzi[ : , 3])
o3d.io.write_point_cloud("./Alldata1.ply", p1)
o3d.visualization.draw_geometries([p1])

In [ ]:
print(intens1[100])

In [ ]:
#Get the normals
normals = p
#Play with radius and nn numbers
o3d.geometry.estimate_normals(normals,search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.2,max_nn=10)) 
o3d.visualization.draw_geometries([normals]) #press n to see the normals

In [ ]:
print(points_xyzi[0])
print(points_xyzi[1])  
print(points_xyzi[2])  
print(points_xyzi[50000])  

In [ ]:
print("Range x")
print(np.max(points_xyzi[:,0]))
print(np.min(points_xyzi[:,0]))
print("Range y")
print(np.max(points_xyzi[:,1]))
print(np.min(points_xyzi[:,1]))
print("Range z")
print(np.max(points_xyzi[:,2]))
print(np.min(points_xyzi[:,2]))